In [19]:
#Creating,visualizing and plotting data about an Ising model after learning about how it works.

In [20]:
#pip install Pillow
#pip install ipywidgets

In [2]:
#Imports

import numpy as np
from math import exp, log, e, sqrt
from ipywidgets import interact
from PIL import Image
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from ipywidgets import interact


In [3]:

N=int(input('What is the length of one side of your grid?')) #Eg 50 makes a 50x50 grid
M=N #Grid is even. playing with an uneven grid might be fun.

J=int(input('What is the Coupling constant? 1 is most common. '))

kT= float(input('What is your Temperature from 0~6? For critical temperature type 22  '))

if kT==22:
    
    kT = 2 / log(1 + sqrt(2), e)#critical temp from Lars Onsangers solution
    
    print('Critical temperature is given as 2 / log(1 + sqrt(2), e), or', kT, 'and will be used for this system. ')


Mag=[]#Places to store magnetization  values
Energy=[] #Places to store energy values


What is the length of one side of your grid?100
What is the Coupling constant? 1 is most common. 1
What is your Temperature from 0~6? For critical temperature type 22  22
Critical temperature is given as 2 / log(1 + sqrt(2), e), or 2.269185314213022 and will be used for this system. 


In [4]:
# ================= create the random spin grid========================

# Each spin has two possible states +1 or −1.


def Initial_Lattice(N,M):
    Spinvalues=[-1,1]
    return np.random.choice(Spinvalues, size=(N, M))




#Here we save it so we can do some calculations from the starting point.

Grid=Initial_Lattice(N,M)

In [5]:
#=========Calculates Magnetization of the grid, which works out to be the summation of every point.======
#If I were to rewrite this code having m= np.sum might lead to quicker times.



def magnetization(Lattice):
    m = 0.0  
    for j in range(0,N):
        for k in range(0,M):  
            m += Lattice[j,k]
  
    return m

-108.0

In [6]:
#========================================= Calculating energy based on formula=================
def Energy_Of_The_System(Lattice):
    E=0.0
 
    
    for i in range(0,N):
        for j in range(0,M):
            
            Spin= Lattice[i, j] #choosen spin
    
    
            SumOfNeighbors= (Lattice[(i+1)%N, (j)]#Right
                     + Lattice[(i-1)%N, (j)]#left
                     + Lattice[(i) , (j+1)%M]#Up
                     + Lattice[(i) , (j-1)%M])#Down
                     
    
            E+=J*Spin*SumOfNeighbors
        
    return E
#Energy_Of_The_System(Grid)

-136.0

In [7]:
#===================Using Energy calculation to find DeltaE and determine spin flips=====================



def EnergyCalc(Lattice, i, j):
    
    
    N, M = Lattice.shape #Giving my Lattice a size
    
    
    Spin= Lattice[i, j] #choosen spin
    
    
    SumOfNeighbors= (Lattice[(i+1)%N, (j)]#Right
                     + Lattice[(i-1)%N, (j)]#left
                     + Lattice[(i) , (j+1)%M]#Up
                     + Lattice[(i) , (j-1)%M])#Down
                     
    
    
    
    dE = (2*J)*(Spin*SumOfNeighbors) #Formula for dE
                   
                   
    if dE <= 0 or exp(-dE / kT) > np.random.random(): #If dE <=0 or boltzmann factor greater than random swtich
        Lattice[i, j] *= -1

In [8]:
#=================================== Movement through the lattice ==============
#This is the simplest case, but experimenting with other iterations over the grid can lead to changed results.

#I've found going randomly across the lattice caused the grid to continue to be random. 
#there might be a proper way to do this, but I've not found it.

#The coordinates of this movement type for a 3x3 Lattice is commented in the cell below if interested. 


def Movement(Lattice):
    N, M = Lattice.shape

    for i in range(N):
        for j in range(M): 
            EnergyCalc(Lattice, i, j)
            
#=========== Calls and saves values for measurements while the grid is being updated============            
            
    Mag.append(magnetization(Lattice))
    Energy.append(Energy_Of_The_System(Lattice))
    return Lattice

In [13]:
#X_Coordinates=[0, 1, 2, 0, 1, 2, 0, 1, 2]
#Y_Coordinates=[0, 0, 0, 1, 1, 1, 2, 2, 2]

In [17]:
# the other way to do this without ipywidgets was just to call 'for i in range(frames)'

In [18]:
#In the following part-
#Used Jake Vanderplas' visualization tutorial https://jakevdp.github.io/blog/2017/12/11/live-coding-cython-ising-model/


#----------------------Visualize the Grid-------------------------------
def Visualize(Lattice):
    return Image.fromarray(np.uint8((Lattice + 1) * 0.5 * 255))  # 0 ... 255 Must be an 8 bit unsigned integer



In [15]:
N_Frames=50

def Evolution(images):
    
    
    def show(frame=(0, len(images)-1)):
        
        
        return Visualize(images[frame])
    
    
    return interact(show)





images = [Grid]


for i in range(N_Frames):
    
    images.append(Movement(images[-1].copy()))
    
Evolution(images);

interactive(children=(IntSlider(value=25, description='frame', max=50), Output()), _dom_classes=('widget-inter…

In [16]:
FrameList=np.arange(0,len(Energy),1)

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "bar"}, {"type": "pie"}],#I didn't change these. These specs just happend to work. Others don't
           ],
)

#Each fig.add_trace is a new graph
#Making traces on the same row and col allow them to be plotted together. 


#traces for Energy and Magnetization
fig.add_trace(go.Scatter(y=(Energy), name='Energy points(J)', mode='lines', marker_color='red'),row=1,col=1)
fig.add_trace(go.Scatter(y=(Mag), name='Magnetization(T)', mode='lines', marker_color='orchid'),row=1,col=1)
fig.update_layout(title='Evolution Data of The Ising Model',
                   xaxis_title='Frames',
                   yaxis_title='Quantity')

#Trace for the table
fig.add_trace(go.Table(
    header=dict(values=['Frame #','Energies', 'Magnetizations'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[FrameList,Energy, Mag], # 2nd column
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left')),row=1,col=2)            

fig.update_layout(width=1000, height=500, showlegend=True)

fig.show()
